In [1]:
import numpy as np
from data import data_video, data_deeplabcut, dataset_opto
from utils import utils
import analysis
import measures
import partial_procrustes_superimposition as pps
import SSM
import os
import pandas as pd
import matplotlib.pyplot as plt
import math
import matplotlib.cm as cm
import pickle
import seaborn as sb
import project

sb.set()

In [2]:
project_path = r'E:\dlc\OpenfieldOpto'
cfg = project.load_project(project_path)

analysis_path = os.path.join(project_path, 'analysis', 'smooth')

In [3]:
dataset = dataset_opto.DatasetOpto(cfg=cfg)

## Create/load project

In [4]:
# project_path = project.create_project(project_path)
project_path = r'E:\dlc\OpenfieldOpto'
cfg = project.load_project(project_path)

In [5]:
dataset = dataset_opto.DatasetOpto(cfg=cfg)

## Choose/calculate reference pose

In [ ]:
# Creates folder with possible reference poses. Then fill in the reference pose of choice in the cfg.yaml file.
# Sessions for reference poses
session_dlc, stim_times = dataset.get_session_dlc('CONTROL', 1248, 1)
session_df = session_dlc.df
session_df = session_df[np.all(session_df.xs('likelihood', level=1, axis=1)>0.9, axis=1)]


In [ ]:
session_df_transl = pps._translate_to_centroid(session_df, cfg['fit_bodyparts'], cfg['impose_bodyparts'])
session_df_transl_rot = pps._rotate_to_xaxis(session_df_transl, cfg['impose_bodyparts'])

In [ ]:
idxs = [500, 1000, 1500, 5000]
fig, axs = plt.subplots(2, 2, figsize=(5, 5), sharex=True)
for i, idx in enumerate(idxs):
    axs[i//2][i%2].scatter(session_df_transl_rot.iloc[idx].xs('x', level=1, axis=0), session_df_transl_rot.iloc[idx].xs('y', level=1, axis=0))
    axs[i//2][i%2].axis('equal')
plt.show()

In [ ]:
mean_pose = np.mean(session_df_transl_rot, axis=0)
filename = os.path.join(cfg['project_path'], 'ref_pose.csv')
pd.DataFrame(mean_pose).T.to_csv(filename)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 3), sharex=True)
ax.scatter(mean_pose.xs('x', level=1, axis=0), mean_pose.xs('y', level=1, axis=0))
ax.axis('equal')
plt.show()

## Preprocess

In [ ]:
analysis.preprocess_all(cfg, dataset, overwrite=False, detect_boxes=False)

## Calculate measures

In [6]:
analysis.calculate_measures(cfg, dataset)

Previously measures data found in Project.
Loading file CAMKII_714.0_1.0.csv ...
nose x   -48.318395
y    43.571962
Name: 0, dtype: float64
tail_start x   -261.241458
y     37.822083
Name: 0, dtype: float64
dot 14270.750354855929
magn 17174.343821305472
nose x   -48.318395
y    43.571962
Name: 0, dtype: float64
tail_start x   -261.241458
y     37.822083
Name: 0, dtype: float64
(2, 19014)
(19014, 31)
(19014,)
Loading file CAMKII_715.0_1.0.csv ...
nose x    7.896394
y    4.498823
Name: 0, dtype: float64
tail_start x   -233.404486
y     13.353878
Name: 0, dtype: float64
dot -1782.9769759387214
magn 2124.6577626021003
nose x    7.896394
y    4.498823
Name: 0, dtype: float64
tail_start x   -233.404486
y     13.353878
Name: 0, dtype: float64
(2, 8015)
(8015, 31)
(8015,)
Loading file CAMKII_722.0_1.0.csv ...
nose x    3.221012
y    5.954641
Name: 0, dtype: float64
tail_start x   -86.515487
y     9.710213
Name: 0, dtype: float64
dot -220.84663394768478
magn 589.385931670305
nose x    3.221012


(37000, 31)
(37000,)
Loading file CONTROL_1022.0_1.0.csv ...
nose x   -219.999217
y   -348.908390
Name: 0, dtype: float64
tail_start x   -330.309802
y   -186.683358
Name: 0, dtype: float64
dot 137803.2878393578
magn 156499.4332905708
nose x   -219.999217
y   -348.908390
Name: 0, dtype: float64
tail_start x   -330.309802
y   -186.683358
Name: 0, dtype: float64
(2, 37000)
(37000, 31)
(37000,)
Loading file GAD2_1074.0_1.0.csv ...
nose x   -8.040237
y    2.539074
Name: 0, dtype: float64
tail_start x   -60.817193
y     5.616132
Name: 0, dtype: float64
dot 503.2444330665203
magn 514.9695651961757
nose x   -8.040237
y    2.539074
Name: 0, dtype: float64
tail_start x   -60.817193
y     5.616132
Name: 0, dtype: float64
(2, 37000)
(37000, 31)
(37000,)
Loading file GAD2_1075.0_1.0.csv ...
nose x    9.759698
y    3.028249
Name: 0, dtype: float64
tail_start x   -4.666988
y    2.555330
Name: 0, dtype: float64
dot -37.810220535210405
magn 54.37129360900626
nose x    9.759698
y    3.028249
Name: 0, dt

(38000, 31)
(38000,)
Loading file CAMKII_1317.0_1.0.csv ...
nose x    4.428140
y    7.314932
Name: 0, dtype: float64
tail_start x   -238.864182
y     15.730880
Name: 0, dtype: float64
dot -942.6537852625179
magn 2046.9109460904417
nose x    4.428140
y    7.314932
Name: 0, dtype: float64
tail_start x   -238.864182
y     15.730880
Name: 0, dtype: float64
(2, 38000)
(38000, 31)
(38000,)
Loading file GAD2_1318.0_1.0.csv ...
nose x   -9.855884
y    5.487024
Name: 0, dtype: float64
tail_start x   -413.470710
y     19.078547
Name: 0, dtype: float64
dot 4179.803969165925
magn 4669.049524665453
nose x   -9.855884
y    5.487024
Name: 0, dtype: float64
tail_start x   -413.470710
y     19.078547
Name: 0, dtype: float64
(2, 37000)
(37000, 31)
(37000,)
Loading file GAD2_1319.0_1.0.csv ...
nose x    13.559898
y     3.220111
Name: 0, dtype: float64
tail_start x   -47.251118
y     2.672723
Name: 0, dtype: float64
dot -632.1138511961336
magn 659.5914307848851
nose x    13.559898
y     3.220111
Name: 0, 

(37000, 31)
(37000,)
nose x     1.072396
y    27.318757
Name: 0, dtype: float64
tail_start x   -276.229563
y     29.833248
Name: 0, dtype: float64
dot 518.7797498368204
magn 7595.977544317618
nose x     1.072396
y    27.318757
Name: 0, dtype: float64
tail_start x   -276.229563
y     29.833248
Name: 0, dtype: float64
(2, 37000)
(37000, 31)
(37000,)
nose x   -8.040237
y    2.539074
Name: 0, dtype: float64
tail_start x   -60.817193
y     5.616132
Name: 0, dtype: float64
dot 503.2444330665203
magn 514.9695651961757
nose x   -8.040237
y    2.539074
Name: 0, dtype: float64
tail_start x   -60.817193
y     5.616132
Name: 0, dtype: float64
(2, 37000)
(37000, 31)
(37000,)
nose x    9.759698
y    3.028249
Name: 0, dtype: float64
tail_start x   -4.666988
y    2.555330
Name: 0, dtype: float64
dot -37.810220535210405
magn 54.37129360900626
nose x    9.759698
y    3.028249
Name: 0, dtype: float64
tail_start x   -4.666988
y    2.555330
Name: 0, dtype: float64
(2, 37000)
(37000, 31)
(37000,)
nose x   -

nose x   -113.604883
y   -122.455120
Name: 0, dtype: float64
tail_start x   -194.387991
y     51.866770
Name: 0, dtype: float64
dot 15732.07337667099
magn 33605.92003636605
nose x   -113.604883
y   -122.455120
Name: 0, dtype: float64
tail_start x   -194.387991
y     51.866770
Name: 0, dtype: float64
(2, 37000)
(37000, 31)
(37000,)
